In [ ]:
from gevent import monkey
monkey.patch_all(thread=False)
from gevent.queue import Queue
import gevent

from IPython.display import clear_output
from datetime import datetime

# = = = = = = = = = = = = = = =

import os
import shutil

if os.path.exists('./file/3.part_code'):
    shutil.rmtree('./file/3.part_code')

os.makedirs('./file/3.part_code')

# = = = = = = = = = = = = = = =

import pandas as pd

# = = = = = = = = = = = = = = =

import requests

import time

import sys
sys.path.append('../00.Tools')
from crawler_configuration import get_header, get_proxy

# = = = = = = = = = = = = = = =

list_file = sorted(list(os.walk('./file/2.page'))[0][2])
while '.DS_Store' in list_file:
    list_file.remove('.DS_Store')
while '0.null.txt' in list_file:
    list_file.remove('0.null.txt')

# = = = = = = = = = = = = = = =

for file in list_file:
    
    # = = = = = = = = = = = = = = =

    output_correct, output_error = pd.DataFrame(), pd.DataFrame()

    # = = = = = = = = = = = = = = =

    input_ = pd.read_excel(f'./file/2.page/{file}',
                           header=0,
                           dtype=str).fillna('')
    
    input_['Product No'] = input_['Product No'].astype(int)
    input_['SKU Count'] = input_['SKU Count'].astype(int)
    input_['Page'] = input_['Page'].astype(int)

    print(f'总数量：{len(input_)}')
    print()

    # = = = = = = = = = = = = = = =


    work = Queue()
    for a in range(len(input_)):
        work.put_nowait(a)

    # = = = = = = = = = = = = = = =

    def crawler():
        global output_correct, output_error

        while not work.empty():
            a = work.get_nowait()

            # = = = = = = = = = = = = = = =

            crawler_status = 'error'

            # = = = = = = = = = = = = = = =

            try:
                if input_.loc[a, 'Country'] == 'US':
                    partnumberauto_string, productnumber_string, name_text, lang, country, lang_2 = 'partnumberauto_en_string', 'productnumber_en_string', 'name_text_en', 'en', 'us', 'en'
                elif input_.loc[a, 'Country'] == 'DE':
                    partnumberauto_string, productnumber_string, name_text, lang, country, lang_2 = 'partnumberauto_de_string', 'productnumber_de_string', 'name_text_de', 'de', 'de', 'de'
                elif input_.loc[a, 'Country'] == 'UK':
                    partnumberauto_string, productnumber_string, name_text, lang, country, lang_2 = 'partnumberauto_en_gb_string', 'productnumber_en_gb_string', 'name_text_en_gb', 'en_gb', 'gb', 'en'
                else:
                    partnumberauto_string, productnumber_string, name_text, lang, country, lang_2 = 'fuck', 'fuck', 'fuck', 'fuck', 'fuck', 'fuck'

                list_part_code, list_type = [], []
                while len(list_part_code) < input_.loc[a, 'SKU Count']:
                    for page in range(input_.loc[a, 'Page']):
                        b = 0
                        while True:
                            b += 1
                            
                            try:
                                request_url = f'''https://www.gates.com/bin/listing/products/variant?draw=1&columns%5B0%5D%5Bdata%5D=&columns%5B0%5D%5Bname%5D={partnumberauto_string}&columns%5B0%5D%5Bsearchable%5D=true&columns%5B0%5D%5Borderable%5D=true&columns%5B0%5D%5Bsearch%5D%5Bvalue%5D=&columns%5B0%5D%5Bsearch%5D%5Bregex%5D=false&columns%5B1%5D%5Bdata%5D=&columns%5B1%5D%5Bname%5D={productnumber_string}&columns%5B1%5D%5Bsearchable%5D=true&columns%5B1%5D%5Borderable%5D=true&columns%5B1%5D%5Bsearch%5D%5Bvalue%5D=&columns%5B1%5D%5Bsearch%5D%5Bregex%5D=false&columns%5B2%5D%5Bdata%5D={name_text}&columns%5B2%5D%5Bname%5D={name_text}&columns%5B2%5D%5Bsearchable%5D=true&columns%5B2%5D%5Borderable%5D=true&columns%5B2%5D%5Bsearch%5D%5Bvalue%5D=&columns%5B2%5D%5Bsearch%5D%5Bregex%5D=false&order%5B0%5D%5Bcolumn%5D=1&order%5B0%5D%5Bdir%5D=asc&start={page * 10}&length=10&search%5Bvalue%5D=&search%5Bregex%5D=false&id={input_.loc[a, 'Product Number']}&lang={lang}&country={country}&fieldList=code_string,{partnumberauto_string},{productnumber_string},{name_text}&filters=&_={int(time.time() * 1000)}'''
                                check_url = f'''https://www.gates.com/{country}/{lang_2}/search.p.{input_.loc[a, 'Product Number']}.html'''
                                
                                resp = requests.get(request_url,
                                                    headers=get_header(),
                                                    proxies=get_proxy(),
                                                    timeout=(10, 10))

                                if resp.status_code == 200:
                                    dict_dict = resp.json()
                
                                    if dict_dict:
                                        break
                            except KeyboardInterrupt:
                                break
                            except:
                                continue
                
                        # = = = = = = = = = = = = = = =

                        for dict_ in dict_dict['data']:
                            if dict_['code_string'] not in dict_:
                                list_part_code.append(dict_['code_string'])
                                list_type.append(dict_['itemtype_string'])

                        # = = = = = = = = = = = = = = =

                        if list_file.index(file) % 3 == 0 and list_file.index(file) != 0 and work.qsize() == 0 or work.qsize() % 100 == 0 and work.qsize() != 0:
                            clear_output()
    
                        print(f'''[尝试次数：{b}] - {(list_file.index(file) + 1) / len(list_file) * 100:.2f}% - {file.removesuffix('.xlsx')} > {input_.loc[a, 'Product No']}.{input_.loc[a, 'Product Name']} > Page {page+1}\n[剩余数量：{work.qsize()}] - [当前时间：{datetime.now().strftime('%H:%M:%S')}]\n''')

                        # = = = = = = = = = = = = = = =
                
                        if len(list_part_code) == input_.loc[a, 'SKU Count']:
                            break

                # = = = = = = = = = = = = = = =

                df_temp = pd.DataFrame({'Product No': input_.loc[a, 'Product No'],
                                        'Product Name': input_.loc[a, 'Product Name'],
                                        'Product Number': input_.loc[a, 'Product Number'],
                                        'Country': input_.loc[a, 'Country'],
                                        'SKU Count': input_.loc[a, 'SKU Count'],
                                        'Page': input_.loc[a, 'Page'],
                                        'No': [i+1 for i in range(len(list_part_code))],
                                        'Type': list_type,
                                        'Part Code': list_part_code})
                
                output_correct = pd.concat([output_correct, df_temp], ignore_index=True).fillna('')

                # = = = = = = = = = = = = = = =

                crawler_status = 'ok'
            except:
                df_temp = pd.DataFrame([input_.iloc[a]]).reset_index(drop=True)
                df_temp.loc[0, 'Request_Url'] = request_url
                df_temp.loc[0, 'Check_Url'] = check_url
            
                output_error = pd.concat([output_error, df_temp], ignore_index=True).fillna('')
                
    # = = = = = = = = = = = = = = =

    list_task = []
    for _ in range(5):
        task = gevent.spawn(crawler)
        list_task.append(task)
    gevent.joinall(list_task)

    print('输出ing...')
    print()
    if not output_correct.empty:
        output_correct = output_correct.sort_values(by=['Product No', 'No'],
                                                    ascending=[True, True],
                                                    ignore_index=True)
        output_correct.to_excel(f'''./file/3.part_code/{file.removesuffix('.xlsx')}-{datetime.now().strftime('%Y%m%d_%H%M%S')}.xlsx''', index=False)
    if not output_error.empty:
        output_error = output_error.sort_values(by=['Product No'],
                                                ascending=[True],
                                                ignore_index=True)
        output_error.to_excel(f'''./file/{file.removesuffix('.xlsx')}-part_code_error.xlsx''', index=False)
        print('爬虫存在error')
        print()

# = = = = = = = = = = = = = = =

list_file = sorted(list(os.walk('./file/3.part_code'))[0][2])
while '.DS_Store' in list_file:
    list_file.remove('.DS_Store')
while '0.null.txt' in list_file:
    list_file.remove('0.null.txt')

# = = = = = = = = = = = = = = =

from tqdm import tqdm
import re

for file in tqdm(list_file, desc='Progress', ncols=77):
    os.rename(f'./file/3.part_code/{file}',
              f'''./file/3.part_code/{re.sub(r'-[0-9]{8}_[0-9]{6}.xlsx$', '.xlsx', file)}''')

print('Done ~')

总数量：1

[尝试次数：1] - 50.00% - crawler_1 > 1.PowerGrip™ Timing Kits > Page 1
[剩余数量：0] - [当前时间：13:36:18]

[尝试次数：1] - 50.00% - crawler_1 > 1.PowerGrip™ Timing Kits > Page 2
[剩余数量：0] - [当前时间：13:36:20]

[尝试次数：1] - 50.00% - crawler_1 > 1.PowerGrip™ Timing Kits > Page 3
[剩余数量：0] - [当前时间：13:36:22]

[尝试次数：1] - 50.00% - crawler_1 > 1.PowerGrip™ Timing Kits > Page 4
[剩余数量：0] - [当前时间：13:36:24]

[尝试次数：1] - 50.00% - crawler_1 > 1.PowerGrip™ Timing Kits > Page 5
[剩余数量：0] - [当前时间：13:36:25]

[尝试次数：1] - 50.00% - crawler_1 > 1.PowerGrip™ Timing Kits > Page 6
[剩余数量：0] - [当前时间：13:36:29]

[尝试次数：1] - 50.00% - crawler_1 > 1.PowerGrip™ Timing Kits > Page 7
[剩余数量：0] - [当前时间：13:36:31]

[尝试次数：1] - 50.00% - crawler_1 > 1.PowerGrip™ Timing Kits > Page 8
[剩余数量：0] - [当前时间：13:36:33]

[尝试次数：1] - 50.00% - crawler_1 > 1.PowerGrip™ Timing Kits > Page 9
[剩余数量：0] - [当前时间：13:36:35]

[尝试次数：2] - 50.00% - crawler_1 > 1.PowerGrip™ Timing Kits > Page 10
[剩余数量：0] - [当前时间：13:36:56]

[尝试次数：2] - 50.00% - crawler_1 > 1.PowerGrip™ Timing 